In [1]:
# Celda para ver cuanta ram tienes

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

print(ram_gb)

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!
89.639657472


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install tensorflow

In [19]:
from PIL import Image

from keras import layers, models
from keras.applications.vgg16 import VGG16
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.regularizers import l2

from skimage import io, color

from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU

import glob
import json
import keras
import keras.backend as K
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import urllib.request

In [7]:
#######################################################
# CONTROL PANEL
#######################################################

target_var = "hdi" # either ("hdi" or "gdp")

In [8]:
# Loading X and Y

features = np.load("/content/drive/MyDrive/Thesis/imageset_500_500.npy") # adjust for colab

if target_var == 'hdi':
    labels = np.load("/content/drive/MyDrive/Thesis/target_HDI.npy") # adjust for colab
    
elif target_var == 'gdp':
    labels = np.load("/content/drive/MyDrive/Thesis/target_logGDP.npy") # adjust for colab

In [9]:
# Split 60% 20% 20%

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.4, random_state=1945, stratify=labels)

# Further split the training set into training and validation sets
X_test, X_val, y_test, y_val = train_test_split(
    X_test, y_test, test_size=0.5, random_state=1945, stratify=y_test)

In [10]:
# Float conversion to allow normalization
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
X_val=X_val.astype('float32')

# Normalization 
X_train=X_train/255.0
X_test=X_test/255.0
X_val=X_val/255.0

In [11]:
# Convert the numerical labels to one-hot encoded format
num_classes = 4
y_train_onehot = keras.utils.to_categorical((y_train-1), num_classes=num_classes)
y_val_onehot = keras.utils.to_categorical((y_val-1), num_classes=num_classes)
y_test_onehot = keras.utils.to_categorical((y_test-1), num_classes=num_classes)

VGG16 

In [ ]:
# Import VGG16
vgg_model = VGG16(include_top=False, input_shape=(500, 500, 3))

# Freezing VGG16 layers
for layer in vgg_model.layers:
    layer.trainable = False

## adding "custom" layers

## Flatten layer
flat_1 = layers.Flatten()(vgg_model.layers[-1].output)

## Dense layers
dense_1 = layers.Dense(256, activation='relu')(flat_1)
dense_2 = layers.Dense(128, activation='relu')(dense_1)
dense_3 = layers.Dense(64, activation='relu')(dense_2)
dense_4 = layers.Dense(32, activation='relu')(dense_3)
dense_f = layers.Dense(16, activation='relu')(dense_4)

#output layer with softmax 
output = layers.Dense(4, activation='softmax')(dense_f)

# define new model
tl_model = Model(inputs=vgg_model.inputs, outputs=output)

# summarize
tl_model.summary()

# compile model
tl_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

# fit model
tl_trained = tl_model.fit(X_train, y_train_onehot, validation_data=(X_val, y_val_onehot), batch_size=64 ,epochs=100, verbose=1)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 500, 500, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 500, 500, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 500, 500, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 250, 250, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 250, 250, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 250, 250, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 125, 125, 128)     0     

KeyboardInterrupt: ignored

ResNet50

In [26]:
# Import ResNet50
resnet_model = ResNet50(include_top=False, input_shape=(500, 500, 3))

# Freezing ResNet layers
for layer in resnet_model.layers:
    layer.trainable = False

## adding "custom" layers

## Flatten layer
flat_1 = layers.Flatten()(resnet_model.layers[-1].output)

## Dense layers
#dense_1 = layers.Dense(16384, activation='relu')(flat_1)
#dense_2 = layers.Dense(2048, activation='relu')(flat_1)
#dense_3 = layers.Dense(500, activation='relu')(flat_1)
dense_4 = layers.Dense(256, activation='relu')(flat_1)
dense_5 = layers.Dense(64, activation='relu')(dense_4)
dense_f = layers.Dense(16, activation='relu')(dense_5)

#output layer with softmax 
output = layers.Dense(4, activation='softmax')(dense_f)

# define new model
tl_resnet_model = Model(inputs=resnet_model.inputs, outputs=output)

# summarize
tl_resnet_model.summary()

# compile model
tl_resnet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

# fit model
tl_resnet_trained = tl_resnet_model.fit(X_train, y_train_onehot, validation_data=(X_val, y_val_onehot), batch_size=64 ,epochs=100, verbose=1)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 500, 500, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 506, 506, 3)  0           ['input_10[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 250, 250, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            